In [ ]:
import joblib
import tkinter as tk
import tweepy as tp
from IPython.display import display, HTML
from trainer import preprocess

fake_tweets = [
    ('Blah blah blah', )
]

MODEL_PATH = r'Models\TFIDF_Logistic Regression_800000_06-13-2023--07-49-51\Logistic Regression.pkl'
VEC_PATH = r'Models\TFIDF_Logistic Regression_800000_06-13-2023--07-49-51\TFIDF.pkl'
model = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VEC_PATH)

def get_tweets(
        username, max_results = 5,
        includingRetweets = False, includingReplies = True
        ) -> tuple[str, str]:
    """Takes a username, a number of results to fetch, and booleans that
    indicate whether or not to include retweets and replies, and returns a
    list of (tweet, id) tuples.
    """
    # needs a working bearer_token to access to Twitter API
    bearer_token = "redacted"
    # These all seem to be unnecessary.
    client_ID = "redacted"
    client_secret = "redacted"
    redirect_uri = "redacted"
    access_token = "redacted"
    access_token_secret = "redacted"
    api_key = "redacted"
    api_secret = "redacted"

    client = tp.Client(bearer_token=bearer_token)

    """client = tp.Client(
        bearer_token=bearer_token, consumer_key=api_key,
        consumer_secret=api_secret, access_token=access_token,
        access_token_secret=access_token_secret
        )"""
    
    ID = client.get_user(username=username).data.id


    def choose_exclusions(rt, rp):
        """Makes a list of things to exclude from get_users_tweets. If
        nothing, returns None instead of an empty list because tweepy does
        not like the empty list."""
        lst = []
        if not rt:
            lst.append('retweets')
        if not rp:
            lst.append('replies')
        if len(lst) == 0: lst = None
        return lst

    tweets = client.get_users_tweets(
        id=ID, max_results=max_results,
        exclude=choose_exclusions(includingRetweets, includingReplies)
        )
    
    temp = []
    for i in range(len(tweets.data)):
        temp.append((tweets.data[i].text, tweets.data[i].id))
    
    tweet_list = temp
    return tweet_list

user = input("Enter username of target account: ")
rt_in = input("Would you like to include retweets? ")
rp_in = input("Would you like to include replies? ")
num = int(input("How many results would you like to include (max 50)? "))
rt = True if 'y' in rt_in.lower() else False if 'n' in rt_in.lower() else None
rp = True if 'y' in rp_in.lower() else False if 'n' in rp_in.lower() else None

tweets_n_ids = get_tweets(user, num, rt, rp)
tweets = list(i[0] for i in tweets_n_ids)
vectorized_docs = vectorizer.transform(preprocess(tweets))
predictions = model.predict(vectorized_docs)

data = []
for i in range(len(predictions)):
    data.append(
        (
        tweets[i],
                f'https://twitter.com/twitter/statuses/{tweets_n_ids[i][1]}',
                predictions[i]
            )
        )

displaying_as = 'html'    

if displaying_as == 'html':
    html = []
    for i in range(len(predictions)):
        html.append(
            f'<h3>{predictions[i]}    <a href="{data[i][1]}">{data[i][0]}</a></h3>'
            )
    html = '\n'.join(html)
    display(HTML(html))

elif displaying_as == 'df':
    pass